# Dashboard 1: Ski Resort Slope Density

Build an Application containing a density mapbox graph that displays color intensity based on `Total slopes`. This map should be centered roughly on the US/Canada.

* Add a slider to this application that filters the resorts mapped to those with a Price below the value selected in the slider.
* Add a radio button that toggles between resorts with night skiing and without night skiing. 
* Consider making your title dynamic based on the price input


In [4]:
from dash import Dash, dcc, html, Input, Output, State, callback, _dash_renderer, no_update, clientside_callback, ALL, Patch
# from dash.exceptions import PreventUpdate
import dash_mantine_components as dmc
from dash_iconify import DashIconify
import plotly.express as px
import plotly.io as pio
import pandas as pd
_dash_renderer._set_react_version("18.2.0")

resorts = (
    pd.read_csv(r"data/resorts.csv", encoding = "ISO-8859-1")
    .query("Country in ['United States', 'Canada']")
)

fig1 = px.density_map(resorts,
                      lat='Latitude',
                      lon='Longitude',
                      z='Total slopes',
                      radius=8,
                      center=dict(lat=45, lon=-104), zoom=2,
                      map_style="light",
                      height=500,
                      title='Ski Resort Slope Density with <b>px.density_map()</b>',
                     )
fig1.add_annotation(
    text="<i><b>px.density_map</b> and <b>go.Densitymap</b> use <b>Maplibre</b> for rendering.\
        <br>These traces were introduced in <b>Plotly.py 5.24</b> and are now the recommended way to make tile-based density heatmaps.</i>",
    showarrow=False,
    yshift=10,
    x=0,
    y=-0.15,
    textangle=0,
    xanchor='left',
    yref="paper",
    xref="paper",
)
fig1.update_annotations(align='left')
fig1

In [5]:
# https://plotly.com/python/density-heatmaps/#mapbox-maps 
# px.density_map and go.Densitymap use Maplibre for rendering.
# These traces were introduced in Plotly.py 5.24. and are now the recommended way to make tile-based density heatmaps.

fig10 = px.density_mapbox(resorts,
                      lat='Latitude',
                      lon='Longitude',
                      z='Total slopes',
                      radius=8,
                      center=dict(lat=45, lon=-104), zoom=2,
                      mapbox_style="open-street-map",
                      height=500,
                      title='Ski Resort Slope Density with <b>px.density_mapbox()</b>'
                     )
fig10.add_annotation(
    text="<i>Plotly is Switching to MapLibre<br><b>https://plotly.com/blog/plotly-is-switching-to-maplibre/</b></i>",
    showarrow=False,
    yshift=10,
    x=0,
    y=-0.15,
    textangle=0,
    xanchor='left',
    yref="paper",
    xref="paper",
)
fig10.update_annotations(align='left')
fig10

# https://plotly.com/blog/plotly-is-switching-to-maplibre/

In [6]:
from dash import Dash, dcc, html, Input, Output, State, callback, _dash_renderer, no_update, clientside_callback, ALL, Patch
# from dash.exceptions import PreventUpdate
import dash_mantine_components as dmc
from dash_iconify import DashIconify

_dash_renderer._set_react_version("18.2.0")

import plotly.express as px
import plotly.io as pio
import pandas as pd
import numpy as np

resorts = (
    pd.read_csv(r"data/resorts.csv", encoding = "ISO-8859-1")
    .query("Country in ['United States', 'Canada']")
)

app = Dash(__name__, external_stylesheets=dmc.styles.ALL)

# dmc.add_figure_templates()
dmc.add_figure_templates(default="mantine_light")

dmc_title = dmc.Title(id='title_id', order=3, 
                      children='',
                      )

# theme_toggle = dmc.ActionIcon(
#     [
#         dmc.Paper(DashIconify(icon="radix-icons:sun", width=25), darkHidden=True),
#         dmc.Paper(DashIconify(icon="radix-icons:moon", width=25), lightHidden=True),
#     ],
#     variant="transparent",
#     color="yellow",
#     id="color-scheme-toggle",
#     size="lg",
#     ms="auto",
#     n_clicks=0,
# )

theme_toggle = dmc.Switch(
    offLabel=DashIconify(icon="radix-icons:sun", width=15, color=dmc.DEFAULT_THEME["colors"]["yellow"][8]),
    onLabel=DashIconify(icon="radix-icons:moon", width=15, color=dmc.DEFAULT_THEME["colors"]["yellow"][6]),
    id="color-scheme-switch",
    persistence=True,
    color="grey",
)

header = dmc.Group([
    dmc_title,
    theme_toggle,
], justify="space-between")

layout = html.Div([
    header,
    html.Br(),
    # html.H2(id='title_id', children='Whatever'),
    dmc.RangeSlider(
        id='price_range_id',
        min=resorts['Price'].min(), # 0
        max=150, # 141
        step=5,
        marks=[{'label': str(val), 'value': val} for val in range(0, 151, 25)],
        value=[50,70],
    ),
    html.Br(),
    dcc.RadioItems(
        id='nigth_option',
        options=[
            {"label": "Has Night Skiing", "value": "Yes"},
            {"label": "No Night Skiing", "value": "No"}],
        value='Yes',
        inline=True,
    ),
    dcc.Graph(
        id='map_id',
        figure={},
    ),
])
app.layout = dmc.MantineProvider(
    [layout],
    id="mantine-provider",
)

@callback(
    Output("map_id", "figure", allow_duplicate=True),
    Input("color-scheme-switch", "checked"),
    prevent_initial_call = True,
)
def update_figure(switch_on):
    # template must be template object rather than just the template string name
    template = pio.templates["mantine_dark"] if switch_on  else pio.templates["mantine_light"]
    patched_figure = Patch()
    patched_figure["layout"]["template"] = template

    return patched_figure


clientside_callback(
    """ 
    (switchOn) => {
       document.documentElement.setAttribute('data-mantine-color-scheme', switchOn ? 'dark' : 'light');  
       return window.dash_clientside.no_update
    }
    """,
    Output("color-scheme-switch", "id"),
    Input("color-scheme-switch", "checked"),
)


@callback(
    Output('title_id', 'children'),
    Output('map_id', 'figure'),
    Input('price_range_id', 'value'),
    Input('nigth_option', 'value'),
)
def update_resorts(price, nigth):
    
    title = f'Resorts with a ticket price between ${price[0]} and ${price[1]}'
    mask_price = resorts['Price'].between(price[0],price[1])
    if nigth == 'Yes':
        mask_nigth = resorts['Nightskiing'].eq('Yes')
    else: mask_nigth = resorts['Nightskiing'].eq('No')
    mask = np.logical_and(mask_nigth, mask_price)
    dff = (resorts[mask])
    fig = px.density_map(dff,
                      lat='Latitude',
                      lon='Longitude',
                      z='Total slopes',
                      radius=8,
                      center=dict(lat=45, lon=-100), zoom=2,
                      map_style="light",
                      height=450,
                     )
    return title, fig


if __name__ == '__main__':
    app.run(debug=True, port=8088)

# Dashboard 2: Top Resorts by Selected Metric

Build an Application that outputs a bar chart of the top 10 resorts based on the dataframe column users select in a dropdown menu. 

* Add a checklist to allow users to select whether they want to limit results to Canada, the US, or include both countries. 
* Add an interactive element that allows users to select whether they want to sort the plotted dataframe in ascending or descending order. 
* Consider making your app title dynamic based on the column selected. 


In [3]:
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

import plotly.express as px
import pandas as pd

resorts = (
    pd.read_csv("../Data/Ski Resorts/resorts.csv", encoding = "ISO-8859-1")
    .query("Country in ['United States', 'Canada']")
)
